In [ ]:
#Initial Setup and importing libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

import time
import glob
import PIL.Image as Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm

#data config
DATA_DIR = 'kaggle/input/vesuvius-challenge-ink-detection/'
BUFFER = 32
Z_DIM = 16
Z_START = 25

BATCH_SIZE = 64

plt.imshow(Image.open(DATA_DIR + "train/1/ir.png"), cmap="gray")